Download http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Dot, Add, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras.regularizers import l2
import keras.backend as K
%matplotlib inline

# Examinando el dataset

In [ ]:
!ls /path/ml-latest-small/

In [ ]:
ratings = pd.read_csv('/path/ml-latest-small/ratings.csv')
print(ratings.shape)
ratings.head(5)

In [ ]:
movie_names = pd.read_csv('/path/ml-latest-small/movies.csv')
movie_names.head()

In [ ]:
# Justamos los 2 datasets y filtramos las columnas que nos interesan
ratings = ratings.join(movie_names, on='movieId', rsuffix='_').loc[:, ['userId', 'movieId', 'rating', 'title']]
ratings.head()

In [ ]:
# Obtenemos la lista de usuarios y peliculas unicas
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [ ]:
# Reemplazamos los IDs por numeros continuos
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

ratings.head()

In [ ]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

# Crear el modelo

In [ ]:
# Cuantos factores va a usar el Embedding para describir nuestras variables
n_factors = 50

# Embeddings para los usuarios
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)

# Embeddings para las peliculas
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

In [ ]:
x = Dot(axes=2)([u, m])
x = Flatten()(x)
model = Model(inputs=[user_in, movie_in], outputs=x)
model.compile(Adam(0.01), loss='mse')
model.summary()

# Entrenar

In [ ]:
# TODO:
# 1. Separar la data en train y validation
# 2. Entrenar la red
# 3. Plotear el entrenamiento
# 4. Ver resultados

# Extra:
# - encontrar un lr adecuado

# Agregar un bias a la red

In [ ]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)

movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

# Bias para los usuarios
u_bias = Embedding(n_users, 1, input_length=1)(user_in)
u_bias = Flatten()(u_bias)

# Bias para las peliculas
m_bias = Embedding(n_movies, 1, input_length=1)(movie_in)
m_bias = Flatten()(m_bias)

x = Dot(axes=2)([u, m])
x = Flatten()(x)

# Sumamos los bias al producto
x = Add()([x, u_bias])
x = Add()([x, m_bias])

model = Model(inputs=[user_in, movie_in], outputs=x)
model.compile(Adam(0.01), loss='mse')
model.summary()

# Explorar los resultados

In [ ]:
# ¿Qué pelicuas tienen el bias mas alto, y cuales el mas bajo?
# ¿Como se puede interpretar los bias de los usuarios y peliculas?

# Extra: Explorar los embeddings
# Dado que los embeddings tienen 50 valores, pueden ser dificiles de explorar.
# Pero podemos usar una reduccion de dimesionalidad, como PCA:
# from sklearn.decomposition import PCA
# - Reducir las dimensionas y visualizar la data